In [214]:
import pandas as pd

In [215]:
df = pd.read_csv("data.csv")
batact = pd.DataFrame()
fin_batact = pd.DataFrame()

In [216]:
df["COMBINED_DT"] = pd.to_datetime(df["DATE"]+' '+df["TIME"])
df["DATE"] = pd.to_datetime(df["DATE"],format="%m/%d/%y")

/tmp/ipykernel_6532/2760743003.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["COMBINED_DT"] = pd.to_datetime(df["DATE"]+' '+df["TIME"])


In [217]:
df.head(5)

,DATE,TIME,STATUS,NAME,TYPE,SOURCE,PRESENT,TECHNOLOGY,CYCLE_COUNT,VOLTAGE_MIN_DESIGN,...,CURRENT_NOW,CHARGE_FULL_DESIGN,CHARGE_FULL,CHARGE_NOW,CAPACITY,CAPACITY_LEVEL,MODEL_NAME,MANUFACTURER,SERIAL_NUMBER,COMBINED_DT
0,2023-07-30,22:57:13,Active,BAT0,Battery,Charging,1,Li-ion,0,11400000,...,13000,3684000,1814000,610000,33,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 22:57:13
1,2023-07-30,22:53:45,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,...,174000,3684000,1814000,610000,33,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 22:53:45
2,2023-07-30,22:08:34,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,...,12000,3684000,1814000,631000,34,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 22:08:34
3,2023-07-30,22:08:21,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,...,3684000,1814000,630000,34,Normal,DELL-FDRHM08,LGC-LGC3.65,34838,NaN,2023-07-30 22:08:21
4,2023-07-30,21:52:34,Active,BAT0,Battery,Charging,1,Li-ion,0,11400000,...,1051000,3684000,1814000,358000,19,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 21:52:34


In [218]:
curr_date = df["DATE"].iloc[-1]
last_date = df["DATE"][0]

In [219]:
rev_df = df[::-1].reset_index(drop=True)

In [220]:
sus_loc = rev_df.loc[rev_df["STATUS"]=="Suspended"]
act_ind = sus_loc.index+1

In [221]:
act_loc = rev_df.loc[act_ind]

In [222]:
sus_loc = sus_loc.reset_index(drop=True)
act_loc = act_loc.reset_index(drop=True)

In [223]:
def format_timedelta(timedelta_object):
    days = timedelta_object.days
    seconds = timedelta_object.seconds
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{24 * days + hours:02d}:{minutes:02d}:{seconds:02d}"


In [224]:
temp_act = pd.DataFrame()
if rev_df["STATUS"][0] == "Active":
    temp_act["DATE"] = pd.Series(rev_df["DATE"][0])
    temp_act["TIMESTAMP"] = pd.Series(pd.to_datetime(sus_loc["COMBINED_DT"][0]) - pd.to_datetime(rev_df["COMBINED_DT"][0]))
    batact["DATE"] = act_loc["DATE"]
    batact["TIMESTAMP"] = pd.to_datetime(act_loc["COMBINED_DT"]) - pd.to_datetime(sus_loc["COMBINED_DT"])
    batact = pd.concat([temp_act,batact],ignore_index=True)

In [225]:
if rev_df["STATUS"][0] == "Suspended":
    batact["DATE"] = act_loc["DATE"]
    batact["TIMESTAMP"] = pd.to_datetime(act_loc["COMBINED_DT"]) - pd.to_datetime(sus_loc["COMBINED_DT"])

In [226]:
batact

,DATE,TIMESTAMP
0,2023-05-03,0 days 00:00:34
1,2023-05-03,0 days 00:00:50
2,2023-05-03,0 days 06:54:16
3,2023-05-03,0 days 00:02:13
4,2023-05-03,0 days 00:00:38
...,...,...
209,2023-07-30,0 days 08:00:33
210,2023-07-30,0 days 00:16:36
211,2023-07-30,0 days 00:01:16
212,2023-07-30,0 days 01:18:50


In [227]:
curr_date=pd.to_datetime(df["DATE"].iloc[-1])
week_d=curr_date+pd.DateOffset(weeks=1)
st_l = [];en_l = []; ch_l = []

In [228]:
while True:
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(batact.loc[(batact["DATE"]>=curr_date)&(batact["DATE"]<=week_d)]["TIMESTAMP"].sum())
    curr_date = curr_date+pd.DateOffset(weeks=1)
    week_d = week_d+pd.DateOffset(weeks=1)
    if(week_d>=batact["DATE"].iloc[-1]+pd.DateOffset(days=1)):
        break

In [229]:
if curr_date<batact["DATE"].iloc[-1]:
    week_d = batact["DATE"].iloc[-1]
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(batact.loc[(batact["DATE"]>=curr_date)&(batact["DATE"]<=week_d)]["TIMESTAMP"].sum())

In [230]:
fin_batact["START_DATE"] = pd.Series(st_l)
fin_batact["END_DATE"] = pd.Series(en_l)
fin_batact["TIME_USED"] = pd.Series(ch_l).apply(format_timedelta)

In [231]:
fin_batact

,START_DATE,END_DATE,TIME_USED
0,2023-05-03,2023-05-10,92:17:35
1,2023-05-10,2023-05-17,106:30:06
2,2023-05-17,2023-05-24,68:38:25
3,2023-05-24,2023-05-31,60:04:08
4,2023-05-31,2023-06-07,90:16:30
5,2023-06-07,2023-06-14,142:44:21
6,2023-06-14,2023-06-21,94:28:58
7,2023-06-21,2023-06-28,129:43:19
8,2023-06-28,2023-07-05,92:13:00
9,2023-07-05,2023-07-12,113:58:38
